In [1]:
import pandas as pd
from pybatfish.client.commands import *
from pybatfish.datamodel import *
from pybatfish.question import bfq, list_questions, load_questions

pd.set_option("display.width", 300) 
pd.set_option("display.max_columns", 20) 
pd.set_option("display.max_rows", 1000) 
pd.set_option("display.max_colwidth", -1)

# Configure all pybatfish loggers to use WARN level
import logging
logging.getLogger('pybatfish').setLevel(logging.WARN)

In [2]:
bf_session.host = 'localhost'

In [3]:
load_questions()


#### Routing and Forwarding Tables

* [Routes](#Routes)
* [Longest Prefix Match](#Longest-Prefix-Match)
* [Prefix Tracer](#Prefix-Tracer)

In [4]:
bf_set_network('generate_questions')

'generate_questions'

In [5]:
bf_set_snapshot('generate_questions')

'generate_questions'

##### Routes

Returns routing tables.

Shows routes for specified RIB, VRF, and node(s).

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | 
network | Examine routes for networks matching this prefix. | str | True | 
protocols | Examine routes for protocols matching this specifier. | [RoutingProtocolSpec](../specifiers.md#routing-protocol-specifier) | True | 
vrfs | Examine routes on VRFs matching this name or regex. | str | True | 
rib | Only return routes from a given protocol RIB. | str | True | 

###### Invocation

In [6]:
result = bfq.routes().answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Node | Node | str
VRF | VRF name | str
Network | Network for this route | str
Next_Hop | Inferred hostname of the next hop | str
Next_Hop_IP | Route's Next Hop IP | str
Next_Hop_Interface | Route's Next Hop Interface | str
Protocol | Route's Protocol | str
Metric | Route's Metric | int
Admin_Distance | Route's Admin distance | int
Tag | Tag for this route | int

Print the first 5 rows of the returned Dataframe

In [7]:
result.head(5)

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,as1border1,default,1.0.1.1/32,None,AUTO/NONE(-1l),GigabitEthernet0/0,local,0,0,None
1,as2dist1,default,2.12.11.0/24,as2core1,2.23.11.2,dynamic,ospf,2,110,None
2,as2core1,default,2.12.21.0/24,None,AUTO/NONE(-1l),GigabitEthernet1/0,connected,0,0,None
3,as2dept1,default,2.128.0.1/32,None,AUTO/NONE(-1l),GigabitEthernet2/0,local,0,0,None
4,as1border2,default,3.0.1.0/24,as3border2,10.13.22.3,dynamic,bgp,50,20,None


Print the first row of the returned Dataframe

In [8]:
result.iloc[0]

Node                  as1border1        
VRF                   default           
Network               1.0.1.1/32        
Next_Hop              None              
Next_Hop_IP           AUTO/NONE(-1l)    
Next_Hop_Interface    GigabitEthernet0/0
Protocol              local             
Metric                0                 
Admin_Distance        0                 
Tag                   None              
Name: 0, dtype: object

In [9]:
bf_set_network('generate_questions')

'generate_questions'

In [10]:
bf_set_snapshot('generate_questions')

'generate_questions'

##### Longest Prefix Match

Returns routes that are longest prefix match for a given IP address.

Return longest prefix match routes for a given IP in the RIBs of specified nodes and VRFs.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
ip | IP address to run LPM on. | str | False | 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | .*
vrfs | Examine routes on VRFs matching this name or regex. | str | True | .*

###### Invocation

In [11]:
result = bfq.lpmRoutes(ip='2.34.201.10').answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Node | Node where the route is present | str
VRF | VRF where the route is present | str
Ip | IP that was being matched on | str
Network | The longest-prefix network that matched | str
Num_Routes | Number of routes that matched (in case of ECMP) | int

Print the first 5 rows of the returned Dataframe

In [12]:
result.head(5)

,Node,VRF,Ip,Network,Num_Routes
0,as2border1,default,2.34.201.10,2.34.201.0/24,2
1,as2border2,default,2.34.201.10,2.34.201.0/24,2
2,as2core1,default,2.34.201.10,2.34.201.0/24,1
3,as2core2,default,2.34.201.10,2.34.201.0/24,1
4,as2dept1,default,2.34.201.10,2.34.201.0/24,1


Print the first row of the returned Dataframe

In [13]:
result.iloc[0]

Node          as2border1   
VRF           default      
Ip            2.34.201.10  
Network       2.34.201.0/24
Num_Routes    2            
Name: 0, dtype: object

In [14]:
bf_set_network('generate_questions')

'generate_questions'

In [15]:
bf_set_snapshot('generate_questions')

'generate_questions'

##### Prefix Tracer

Traces prefix propagation through the network.

Shows how prefixes are treated by devices in the network during routing.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Include prefix tracing information for nodes matching this name or regex. | [NodeSpec](../specifiers.md#node-specifier) | True | 
prefix | The prefix to trace. Expected format is A.B.C.D/Y. | str | True | 

###### Invocation

In [16]:
result = bfq.prefixTracer(prefix='2.34.201.0/24').answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Node | The node where action takes place | str
VRF | The VRF where action takes place | str
Peer | The node's neighbor to which the action applies | str
Action | The action that takes place | str
Prefix | The prefix in question | str

Print the first 5 rows of the returned Dataframe

In [17]:
result.head(5)

,Node,VRF,Peer,Action,Prefix
0,as2border2,default,as3border1,filtered_out,2.34.201.0/24
1,as2dept1,default,as2dist2,filtered_out,2.34.201.0/24
2,as2dist2,default,as2dept1,filtered_out,2.34.201.0/24
3,as2border1,default,as1border1,filtered_out,2.34.201.0/24
4,as2core2,default,as2dist2,filtered_out,2.34.201.0/24


Print the first row of the returned Dataframe

In [18]:
result.iloc[0]

Node      as2border2   
VRF       default      
Peer      as3border1   
Action    filtered_out 
Prefix    2.34.201.0/24
Name: 0, dtype: object